## Querying Data

In order to query data you will need:
* Installed DQ0 SDK. Install with `pip install dq0-sdk`
* Installed DQ0 CLI.
* Proxy running and registered from the DQ0 CLI with `dq0-cli proxy add ...`
* Valid session of DQ0. Log in with `dq0 user login`
* Running instance of DQ0 CLI server: `dq0 server start`

* DQ0 Project with Data attached to it.
Keep in mind that a query is always executed within the context of a project.

Start by importing the core classes

In [1]:
%cd ../

/Users/dominic/Projects/dq0-sdk


In [2]:
# import dq0-sdk api
from dq0.sdk.cli import Project, Data, Query

## Create or load a project
Projects act as the working environment for model development.
Each project has a model directory with a .meta file containing the model uuid, attached data sources etc.
Creating a project with `Project.create(name='model_1')` is equivalent to calling the DQ0 Cli command `dq0-cli project create model_1`.
Alternatively, if you want to load an existing project, navigate your current working directory to the project directory so you can use the Project.load() method.

In [3]:
# create a project with name 'model_1'. Automatically creates the 'model_1' directory and changes to this directory.
# project = Project(name='project_1')

## Load a project
Alternatively, you can load an existing project by first cd'ing into this directory and then call Project.load()
This will read in the .meta file of this directory.

In [22]:
%cd ../dq0-cli/MyNewProject

[Errno 20] Not a directory: '../dq0-cli/MyNewProject'
/Users/dominic/go/src/dq0-cli/MyNewProject


In [5]:
# Alternative: load a project from the current model directory
project = Project.load()

Check if the project was successfully set up by checking it's UUID. If this field is empty, something went wrong.

In [6]:
project.project_uuid

'04a74fa8-8af0-4fb5-82c5-06e6966702fe'

## Get and attach data source
For new projects you need to attach a data source. Existing (loaded) projects usually already have data sources attached.

In [7]:
# first get some info about available data sources
sources = Data.get_available_data_sources()

# get info about the first source
info = Data.get_data_info(sources[1])
info

{'commit_uuid': '360ac136-c6a1-4ff6-9328-37b66ebb23c4',
 'data_uuid': 'b1ed3b6c-1a71-4597-98ec-6583e1b4ee99',
 'data_name': 'cpg_segments',
 'data_type': 'CSV',
 'data_description': 'some description',
 'privacy_budget': {'initial': 1000,
  'current': 994,
  'created_at': 1610619263,
  'updated_at': 1610622299},
 'data_usage': 89,
 'data_privacy_column': 'idl',
 'data_size': 1000,
 'data_meta': 'bmFtZTogY3BnX3NlZ21lbnRzCmRlc2NyaXB0aW9uOiBzb21lIGRlc2NyaXB0aW9uCnR5cGU6IENTVgpjb25uZWN0aW9uOiBmaWxlOi8vL1VzZXJzL2RvbWluaWMvUHJvamVjdHMvZHEwLXNxbC90ZXN0cy9kYXRhL2NwZ19zZWdtZW50cy5jc3YKcHJpdmFjeV9idWRnZXQ6IDEwMDAKcHJpdmFjeV9idWRnZXRfaW50ZXJ2YWxfZGF5czogMzAKc3ludGhfYWxsb3dlZDogZmFsc2UKcHJpdmFjeV9sZXZlbDogMgpwcml2YWN5X2NvbHVtbjogaWRsCnNpemU6IDEwMDAKTFI6CiAgY3BnX3NlZ21lbnRzOgogICAgcm93X3ByaXZhY3k6IGZhbHNlCiAgICByb3dzOiAxMDAwCiAgICBtYXhfaWRzOiAxCiAgICBzYW1wbGVfbWF4X2lkczogdHJ1ZQogICAgY2Vuc29yX2RpbXM6IHRydWUKICAgIGNsYW1wX2NvdW50czogZmFsc2UKICAgIGNsYW1wX2NvbHVtbnM6IHRydWUKICAgIGlkbDoKICAgICAgcHJpdmF0Z

In [8]:
# print information about column types and values, description. This may be helpful for creating your queries.
info['data_name']

'cpg_segments'

In [9]:
info['data_type']

'CSV'

In [10]:
info['data_description']

'some description'

Suppose are happy with this information and now want to query this dataset in our project.

In [11]:
# set data
data = sources[1]

# alternatively, if you already know the name of the dataset:
# data = Data('name_of_dataset')

### New projects
For new projects, we need to attach this data source first.

In [12]:
project.attach_data_source(data=data)

data is already attached to project


### Existing projects
For existing projects, we need to check whether the dataset of interest is already attached to our project.

In [13]:
project.get_attached_data_sources()

## Create Query

Once we have a project with data attached to it we can create our query. Think of this object like a query manager that can create multiple query runs.

In [14]:
query = Query(project)

Now we can use this Query instance to start the actual query runs. But first we must specify which datasets we want to query:

In [15]:
query.for_data(data)

Prepare your query statement.

In [16]:
stmt = """SELECT SUM(active_complaint), COUNT(*) as tx_count, c.loyalty_tiers FROM LR.cpg_segments as c WHERE c.loyalty_tiers = 'silver' AND c.active_complaint > 0 GROUP BY loyalty_tiers ORDER BY tx_count DESC LIMIT 600"""

## Execute query
We can now pass this statement to execute() method, which returns a new QueryRunner instance. We will use this to check our queries progress/state and results. Keep in mind that queries are executed asynchronously. 

In [17]:
run = query.execute(stmt)

In [18]:
# check status
run.get_state()

# Or wait for the query to finish - careful, this may take a while!
run.wait_for_completion(verbose=True)

# Once its finished, we can get the results
result = run.get_results()
print(result)

running
Waiting for job to complete...
finished
finished
Job completed
finished
0x7be3,tx_count,c_loyalty_tiers
2556,2561,silver



#### Displaying Errors

Of course, not all of your queries will finish without errors. If get_state returns an 'error', call the .get_error() method to show some more details:

In [19]:
run2 = query.execute('foo')
run2.wait_for_completion()
run2.get_error()

running
error
2021-01-19T15:50:55Z | dq0.sql.runner | INFO | [__KEYWORD_STARTED__] Started with args: Namespace(_loglevel='debug', epsilon=1.0, meta=['bmFtZTogY3BnX3NlZ21lbnRzCmRlc2NyaXB0aW9uOiBzb21lIGRlc2NyaXB0aW9uCnR5cGU6IENTVgpjb25uZWN0aW9uOiBmaWxlOi8vL1VzZXJzL2RvbWluaWMvUHJvamVjdHMvZHEwLXNxbC90ZXN0cy9kYXRhL2NwZ19zZWdtZW50cy5jc3YKcHJpdmFjeV9idWRnZXQ6IDEwMDAKcHJpdmFjeV9idWRnZXRfaW50ZXJ2YWxfZGF5czogMzAKc3ludGhfYWxsb3dlZDogZmFsc2UKcHJpdmFjeV9sZXZlbDogMgpwcml2YWN5X2NvbHVtbjogaWRsCnNpemU6IDEwMDAKTFI6CiAgY3BnX3NlZ21lbnRzOgogICAgcm93X3ByaXZhY3k6IGZhbHNlCiAgICByb3dzOiAxMDAwCiAgICBtYXhfaWRzOiAxCiAgICBzYW1wbGVfbWF4X2lkczogdHJ1ZQogICAgY2Vuc29yX2RpbXM6IHRydWUKICAgIGNsYW1wX2NvdW50czogZmFsc2UKICAgIGNsYW1wX2NvbHVtbnM6IHRydWUKICAgIGlkbDoKICAgICAgcHJpdmF0ZV9pZDogdHJ1ZQogICAgICB0eXBlOiBzdHJpbmcKICAgIGFjdGl2ZV9jb21wbGFpbnQ6CiAgICAgIHR5cGU6IGludAogICAgbG95YWx0eV90aWVyczoKICAgICAgdHlwZTogc3RyaW5nCg=='], metapath=[], param=None, private_column='idl', query='foo', tau=None, tracker_group_uuid='38', tracke

#### Setting query parameters
The above run uses the default parameters defined by the execute() method for this query. These are:

    * epsilon: float; Epsilon value for differential private query. Default: 1.0
            
    * tau: float; Tau threshold value for private query. Default: 0.0
            
    * private_column: string; Private column for this query. Leave empty or omit for default value from metadata.

Naturally we can adjust these:

In [20]:
run3 = query.execute(stmt, epsilon=1.5, tau=100, private_column='idl')
run3.wait_for_completion(verbose=True)
run3.get_results()
# the results are now also stored in run2.state.results

Waiting for job to complete...
running
running
finished
finished
Job completed
finished


'0x4e9b,tx_count,c_loyalty_tiers\n2556,2548,silver\n'

#### Visualizing results
The get_results() method returns the raw result payload as a string. Usually, this payload comes in CSV format. Here we pandas to display this data.

In [21]:
import pandas as pd
from io import StringIO

result_str = run3.state.results

df = pd.read_csv(StringIO(result_str))
df

,0x4e9b,tx_count,c_loyalty_tiers
0,2556,2548,silver
